### Optimisation of a Bioprocess with Multifidelity Bayesian Optimisation


#### Hackathon Breif
This hackathon involves the optimisation of a simulated bioprocess at process scale utilising CHO cells to produce a desired protein. Experimentally, this would involve a resource-intensive screening campaign involving the growth and feeding of cells under precise conditions (temperature, pH, feed amount, fidelity, etc.) to maximize the production of a desired product. This hackathon offers a simulated method of mapping bioprocess input parameters to a final predicted titre concentration: a measure of cell productivity. The simulations are based on various kinetic parameters which are unique to the type of cells used. For the final scoring, a different set of cell kinetic parameters will be used to evaluate your algorithm. 

#### Inputs and Outputs
Inputs to the bioprocess includes 5 vairables: the temperature [°C], pH and the concentration of feed [mM] at 3 different timepoints over 150 minutes. The output is the concentration of the titre (desired product) [g/L]. The goal is to obtain the input variables that correspond to the highest obtained titre. 

The bounds of the inputs are as follows: 

```
temperature [°C]               -> 30 - 40
pH                             -> 6 - 8
first feed concentration [mM]  -> 0 - 50
second feed concentration [mM] -> 0 - 50
third feed concentration [mM]  -> 0 - 50
```

#### Fidelities and Running the simulation
The simulations can be perfomed at 3 levels of fidelities with an associated accuracy and costs. These fidelities corresponds to a different reactor type and scale used. 

```
Lowest fideility: 3L reactor with 1 feeding timepoint at 60 mins.
Realtive cost: 10
Remarks: The feeding concentration is taken as the second feed concentration. Lowest accuracy, but also lowest cost. 

Middle fidelity: 3L reactor with 3 feeding timepoints at 40, 80, 120 mins.
Relative cost: 575
Remarks: -

Highest fidelity: 15L reactor with 3 feeding timepoints at 40, 80, 120 mins.
Relative cost: 2100
Remarks: Highest accuracy but high cost.
```

To run an experiment, one can use the `conduct_experiment(X)` function -> this is your objective function. The inputs to this function is a matrix of shape (N, 6) where N is the number of data points and 6 refers to the total number of variables in the following order: `[temperature, pH, feed1, feed2, feed3, fidelity]`. The fidelities are refered to as integers where `0` corresponds to the lowest fidelity, `1` with the middle and `2` with the highest fidelity. An example is shown below. 

``` python
import numpy as np
def obj_func(X):
	return (-np.array(conduct_experiment(X))) #negative placed if optimisation performed is minimisation

X_initial = np.array([[33, 6.25, 10, 20, 20, 0],
                      [38, 8, 20, 10, 20, 0]])
Y_initial = conduct_experiment(X_initial)
print(Y_initial)
```

#### Goal and Submission
Your goal is to develop a Bayesian Optimisation class to obtain the set of inputs which maximizes the titre. You have a **budget of 10000** (observe the cost of running each fidelity), a maximum runtime (on the intructor's computer - be aware of how large the search space becomes especially with 6 dimensions!) and starting with a maximum of 6 training points. (Remember, you have to have at least 2 points for each variable for the covariance matrix to be calculated.)

Please submit your BO class (and GP class) along with the execution block as a .py file to the instructor. A different cell type (with different simulation parameters and maxima) will be used for scoring.

This hackathon will be scored based on the sum of the titre concentration obtained. The score will be penalised by your algorithm's runtime in seconds. 

You must stay within the allocated budget! This will be checked, and if exceeded, your submission will be disqualified!

#### Form of the BO class and execution block
You are allowed to write your own BO class or make modifications to any of the previously seen BO classes. 

You must include the attributes `self.X` and `self.Y` corresponding to all of your evaluated inputs and outputs as this will be used to retrive the information used for scoring. 

```python
#submission should look something like the following
class GP: #if you have any separate classes other than the BO class
    def __init__(self, ...):
        ...
#BO class
class BO: 
    def __init__(self, ...):
        self.X = #training data which the evaluated data is to be appended
        self.Y = #evaluated via the objective function using self.X

# BO Execution Block
X_training = [...]
X_seachspace = [...]

BO_m = BO(...)
```

#### Guidance (Advanced)
You can develop a multifidelity Bayesian Optimisation algorithm. Your scoring will be additionally penalised by the code runtime in the units of seconds. Make your algorithm as fast as it can go!


#### Guidance (Intermediate) 
You do not have to develop a multifidelity BO algorithm. You could, if you choose, use the single or batch BO algorithm to perform the optimisation. The lowest fidelity experiments do not offer accurate outcomes and you have to choose how many number of expeirments for each fidelity to be performed such that you do not exceed your allocated budget. 

However, if you do wish to tackle the hackathon via a multifidelity BO by modifying the single batch BO code from the first hackathon. Here are some pointers. 
1. Observe the output of the `vl.conduct_experiment(X)` function. The output does not have the same array structure/shape as the outputs obtained in the previous sections. You have to modify this in order to accomodate for the BO algorithm.
2. Create a new acquisition function that is cost aware. We have previously used Lower Confidence Bound to balance exploration and exploitation of the search space. To make this cost aware, we can scale the values obtained from LCB by the cost.

```python
    def MF_lower_confidence_bound(...):
        lower_std = Ysearchspace_mean - acquisition_hyperparam[0]*np.sqrt(Ysearchspace_std)
        # mf_lower_std = lower_std / assocated cost for each simulation
        return (X_searchspace[np.argmin(mf_lower_std)])
```

If this is done succefully, well done! However, you might see that the code will run rather slowly for each iteration (remember how the runtime scales with respect to additional dimensions in the search space). If you are finding it difficult to run the full iterations, a recommendation is to lower the number of total points in your search space. For example, if you are using the np.linspace() function, start with a very course number of points for each dimension (ex. 3) to develop your code. Once you are happy that the code can run without errors, then you can increase the number of points per dimension. 

#### Package Imports

Packages are limited to the the ones listed in the package cell - Talk to one of the intructors to ask if it is possible to import other packages

In [ ]:
# if using google collab, run the following pip installs!
!pip install sobol_seq
!pip install plotly
!pip install gpytorch
!pip install rdkit
!pip install botorch


In [ ]:
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D
import plotly.graph_objs as go
from scipy.integrate import quad
from scipy.spatial.distance import cdist
from scipy.optimize import minimize, differential_evolution, NonlinearConstraint
from sklearn.decomposition import PCA
import math
import time
import sobol_seq
import torch
import gpytorch
import copy
import numpy as np

In [26]:
import pandas as pd
reactor_list = ["3LBATCH", "3LCONTBATCH", "15LCONTBATCH"]
process_parameters = {
    "3LBATCH": {
        "celltype_1": {"my_max": 0.035, "K_lysis": 4e-2,   "k": [1e-3, 1e-2, 1e-2],          "K": [150, 40, 1, 0.22],    "Y": [9.23e7, 8.8e8, 1.6, 0.68, 6.2292e-8, 4.41e-6],    "m": [8e-13, 3e-12], "A": 1e1, "pH_opt": 7.2, "E_a": 32},
        
    },
    "3LCONTBATCH": {
        "celltype_1": {"my_max": 0.035, "K_lysis": 4e-2,   "k": [1e-3, 1e-2, 1e-2],          "K": [150, 40, 1, 0.22],    "Y": [9.23e7, 8.8e8, 1.6, 0.68, 6.2292e-8, 4.41e-6],    "m": [8e-13, 3e-12], "A": 1e1, "pH_opt": 7.2, "E_a": 32},

    },
    "15LCONTBATCH": {
        "celltype_1": {"my_max": 0.035, "K_lysis": 4e-2,   "k": [1e-3, 1e-2, 1e-2],          "K": [150, 40, 1, 0.22],    "Y": [9.23e7, 8.8e8, 1.6, 0.68, 6.2292e-8, 4.41e-6],    "m": [8e-13, 3e-12], "A": 1e1, "pH_opt": 7.2, "E_a": 32},
        
    }
}
NOISE_LEVEL = {
            "3LBATCH": 2e-1,    
            "3LCONTBATCH": 8e-2,
            "15LCONTBATCH": 8e-5
        }
fidelity_cost = {
            "3LBATCH": 0.05,    
            "3LCONTBATCH": 0.5,
            "15LCONTBATCH": 1
        }
data = []
for reactor, cell_data in process_parameters.items():
    for cell_type, params in cell_data.items():
        entry = {
            "reactor": reactor,
            "cell_type": cell_type,
            **params
        }
        data.append(entry)
df = pd.DataFrame(data)


import numpy as np
from scipy.integrate import solve_ivp
# import C_Bioprocess_Utils.conditions_data as data
import pandas as pd


class EXPERIMENT:
    def __init__(
            self, 
            T: float = 32, 
            pH: float = 7.2, 
            cell_type: str = "celltype_1", 
            reactor: str = "3LBATCH", 
            feeding: list = [(10, 0), (20, 0), (30, 0)], 
            time=150, 
            df =df
        ):

        df = df
        params = df[(df['reactor'] == reactor) & (df['cell_type'] == cell_type)]
        self.reactor    = reactor
        self.volume     = 3 
        self.cell_type  = cell_type
        self.time       = time 
        self.my_max     = params["my_max"].iloc[0]                      
        self.K_lysis    = params["K_lysis"].iloc[0]
        self.K_L, self.K_A, self.K_G, self.K_Q = params["K"].iloc[0]
        self.Y = params["Y"].iloc[0] 
        self.m = params["m"].iloc[0] 
        self.k_d_Q, self.k_d_max, self.k_my = params["k"].iloc[0]

        self.A      = params["A"].iloc[0] 
        self.E_a    = params["E_a"].iloc[0] 
        self.pH_opt = params["pH_opt"].iloc[0] 

        self.initial_conditions = [0, 1e6, 0.8 * 1e6, 0, 210, 1, 9, 0]
        self.solution = None 
        self.t = None       
        self.T         = T  
        self.pH        = pH 

        self.feeding = feeding
        
        self.R = 8.314  

    def temperature_effect(self):
        x = self.T
        mu = self.E_a
        A = 5

        left_part = np.exp(-1 * ((x - mu) / 10)**2)
        right_part = np.exp(-0.9 * ((x - mu) / 3.6)**2)  

        factor = A * np.where(x < mu, left_part, right_part)
        return factor
    

    def pH_effect(self) -> float:
        x = self.pH
        mu = self.pH_opt
        A = 2

        left_part = np.exp(-0.8 * ((x - mu) / 1)**2)
        right_part = np.exp(-1 * ((x - mu) / 0.5)**2) 

        factor = A * np.where(x < mu, left_part, right_part)
        return factor

    def my(self, G, Q, L, A):
        temperature_factor = self.temperature_effect()
        pH_factor = self.pH_effect()

        my_max = self.my_max
        K_G = self.K_G
        K_Q = self.K_Q
        K_L = self.K_L
        K_A = self.K_A

        my = my_max * G/(K_G + G) * Q/(K_Q + Q) * K_L/(K_L + L) * K_A/(K_A + A) * temperature_factor * pH_factor
        return my

    def ODE(self,t,x):
        P, X_T, X_V, X_D, G, Q, L, A = x
        my = self.my(G, Q, L, A)
        k_d = self.k_d_max * (self.k_my/(my + self.k_my))
        K_lysis = self.K_lysis
        k_d_Q = self.k_d_Q
        K_G = self.K_G
    
        Y_X_G, Y_X_Q, Y_L_G, Y_A_Q, Y_P_X, Y_dot_P_X = self.Y
        m_G, m_Q = self.m
        
        dX_T_dt = my * X_V - K_lysis * X_D
        dX_V_dt = (my-k_d) * X_V
        dX_D_dt = k_d * X_V - K_lysis * X_D

        dP_dt = Y_P_X * X_T + Y_dot_P_X * (my * G / (K_G + G)) * X_V

        dG_dt = X_V * (-my/Y_X_G - m_G)
        dQ_dt = X_V * (-my/Y_X_Q - m_Q) - k_d_Q * Q
        dL_dt = -X_V * Y_L_G * (-my/Y_X_G - m_G)
        dA_dt = -X_V * Y_A_Q * (-my/Y_X_Q - m_Q) + k_d_Q * Q

        gradients = [dP_dt, dX_T_dt, dX_V_dt, dX_D_dt, dG_dt, dQ_dt, dL_dt, dA_dt]

        return gradients

    def ODE_solver(self):
        t_span = (0, self.time)  
        t_eval_total = [] 
        y_total = []      
        current_t = 0      
        current_y = self.initial_conditions.copy() 

        for event_time, new_G_value in self.feeding:
            t_span_segment = (current_t, event_time)
            t_eval_segment = np.linspace(current_t, event_time, 1000)
            
            solution = solve_ivp(
                fun=self.ODE,
                t_span=t_span_segment,
                y0=current_y,
                t_eval=t_eval_segment,
                method="RK45"
            )

            t_eval_total.extend(solution.t)
            y_total.append(solution.y)
            
            current_t = event_time
            current_y = solution.y[:, -1]
            if current_y[4] < new_G_value:
                current_y[4] = new_G_value 

            new_Q_value = new_G_value * 0.4
            if current_y[5] < new_Q_value:
                current_y[5] = new_Q_value 

        t_span_segment = (current_t, self.time)
        t_eval_segment = np.linspace(current_t, self.time, 500)
        
        solution = solve_ivp(
            fun=self.ODE,
            t_span=t_span_segment,
            y0=current_y,
            t_eval=t_eval_segment,
            method="RK45"
        )
        
        t_eval_total.extend(solution.t)
        y_total.append(solution.y)

        t_eval_total = np.array(t_eval_total)
        y_total = np.hstack(y_total)
        
        self.solution = y_total
        self.solution[0] = self.solution[0] / (self.volume * 1e3) # Transform unit into g/L

        self.t = t_eval_total
        return y_total
 
    def measurement(self, noise_level=None, quantity="P"):
        # NOTE: this makes every call produce the same noise
        # If you want different noise each call, remove this line.
        np.random.seed(1234)

        reactor_type = self.reactor

        # noise_level can be:
        #  - None (use default map)
        #  - dict (map per reactor)
        #  - number (use directly)
        if noise_level is None:
            noise_level = NOISE_LEVEL.get(reactor_type, None)
            if noise_level is None:
                raise ValueError(f"Unknown reactor type: {reactor_type}")
        elif isinstance(noise_level, dict):
            noise_level = noise_level.get(reactor_type, None)
            if noise_level is None:
                raise ValueError(f"Unknown reactor type in provided noise map: {reactor_type}")
        else:
            noise_level = float(noise_level)

        self.ODE_solver()

        index = {"P": 0, "X_T": 1, "X_V": 2, "X_D": 3, "G": 4, "Q": 5, "L": 6, "A": 7}
        true_value = self.solution[index[quantity]][-1]

        noise_magnitude = max(noise_level * true_value, 1e-8)
        noise = np.random.normal(0, noise_magnitude)
        return true_value + noise


def conduct_experiment(X, initial_conditions: list = [0, 0.4 * 1e9, 0.4 * 1e6, 0, 20, 3.5, 0, 1.8], noise_level=None):
    result = []
    feeding = [(10, 0), (20, 0), (30, 0)]
    reactor = "3LBATCH"

    for row in X:
        if len(row) == 2:
            T, pH = row
        elif len(row) == 5: 
            T, pH, F1, F2, F3 = row
            feeding = [(40, float(F1)), (80, float(F2)), (120, float(F3))]
        elif len(row) == 6: 
            T, pH, F1, F2, F3, fidelity = row
            if np.round(fidelity) == 0:
                feeding = [(40, 0), (60, float(F2)), (120, 0)] 
            else:
                feeding = [(40, float(F1)), (80, float(F2)), (120, float(F3))]
            reactor = reactor_list[int(np.round(fidelity))]
        else:
            raise ValueError(f"Cannot handle the dimensionality of X. n must be 2, 5 or 6 but is {len(row)}")

        cell = EXPERIMENT(T=T, pH=pH, time=150, feeding=feeding, reactor=reactor)
        cell.initial_conditions = initial_conditions
        value = float(cell.measurement(quantity="P", noise_level=noise_level))
        #print(value)
        result.append(value)
    return result

In [29]:
# Check if this runs without errors!

def obj_func(X):
	return (-np.array(conduct_experiment(X))) #negative placed if optimisation performed is minimisation

X_initial = np.array([[33, 6.25, 10, 20, 20, 0],
                      [38, 8, 20, 10, 20, 0]])
Y_initial = conduct_experiment(X_initial)
print(Y_initial)

[10.35526663386677, 1.3630867118273524]
